# *RUN THE FIRST TWO CHUNKS*

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import r2_score

#### Import LOB, LOD, LOQ values

In [89]:
rpath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LOD_calcs.csv"
results = pd.read_csv(rpath)

#### Import compositions

In [4]:
cpath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LANL cal_comps\\LANL_rdr_cal2015_1600mm_pls_metadata_060921.csv"
comps = pd.read_csv(cpath)

# First/reset calculations?
#### Set up results table

In [127]:
full_table = pd.DataFrame({
    "element" : [],
    'instrument' : [],
    'atmosphere' : [],
    'num_range' : [],
    "Avg_Braga" : [],
    "Avg_metals" : [],
    "RMSEP_Braga" : [],
    "RMSEP_metals" : [],
    "R2_Braga" : [],
    "R2_metals" : []
})

# Adding new data?
#### Import existing table

In [ ]:
#fpath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\test_calcs.csv"
#full_table = pd.read_csv(fpath)

# \*\*\*Start here with a new calculation\*\*\*
Data should be in a `XX_preds.csv` or `XX_predictions.csv` format within folders separated by random number region (0-750 or 250-1000)

- **LANL 0-750:** C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LANL calculations\\models\\0-750\\
- **LANL 250-1000:** C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\LANL calculations\\models\\250-1000\\

In [130]:
instrument = input("What instrument was the data collected on? ")
atm = input("What atmosphere was the data collected under? ")
num_range = input("What range of random numbers were the models trained on? (0-750 or 250-1000)")
folder = input('Folder path containing test predictions:')
predlist = os.listdir(folder)

What instrument was the data collected on? LANL
What atmosphere was the data collected under? Mars
What range of random numbers were the models trained on? (0-750 or 250-1000)0-750
Folder path containing test predictions:C:\Users\ytsma22c\Google Drive\Misc Work\LOD paper\LANL calculations\models\0-750\


In [131]:
# subselect the relevant parts of the results table
new_results = results[(results.instrument == instrument) & 
                      (results.atmosphere == atm) &
                      (results.num_range == num_range)]

In [132]:
# prepare lists to be filled
# ADD LISTS FOR 'ALL' AND 'ABOVE LOD' CALCULATIONS IF DESIRED
elem = []
avg_braga = []
rmsep_braga = []
r2_braga = []
avg_metal = []
rmsep_metal = []
r2_metal = []

In [133]:
for file in tqdm(predlist):
    if "test" in file:
        #get path name        
        path = (folder + file)
        #read in file
        data = pd.read_csv(path)
        #get element name, add to list
        element = data.columns[1]
        elem.append(element)
        # format columns
        data.columns = ['pkey', 'Actual', 'Pred']
        data = data.drop([0])
        data.Pred = data.Pred.astype(float)       
        # remove predictions above 100 for majors
        if element in ['SiO2', 'MnO', 'Na2O']:
            data = data[data.Pred < 100]
        # remove all predictions below 0
        data = data[data.Pred > 0].reset_index(drop=True)
        # subselect relevant reference values
        ref = new_results[new_results.element == element].reset_index(drop=True)

        # add in Actual concentrations
        temp_comps = comps[comps.pkey.isin(data.pkey)].reset_index(drop=True)
        data['Actual'] = temp_comps[temp_comps['pkey'] == data['pkey']][element]

#### COULD ADD CODE HERE TO CALCULATE AVERAGES, RMSE-PS, AND R2S FOR #####
####### ALL VALUES AND FOR THOSE ABOVE THE LOD BUT BELOW THE LOQ #########
        
        ###BRAGA###
        loq_b = ref['LOQ_Braga'].iloc[0]
        # select just predictions above the LOQ
        braga = data[data.Pred > loq_b].reset_index(drop=True)
        # get average concentration
        avg_b = braga['Actual'].mean()
        avg_braga.append(avg_b)
        # get R2
        r2_b = r2_score(braga.Actual, braga.Pred)
        r2_braga.append(r2_b)
        # get RMSE-P
        braga['sqerror'] = (braga.Actual - braga.Pred).pow(2)
        rmse_b = braga['sqerror'].mean() ** 0.5
        rmsep_braga.append(rmse_b)
        
        ##METALS###
        loq_m = ref['LOQ_metals'].iloc[0]
        # select just predictions above the LOQ
        metal = data[data.Pred > loq_m].reset_index(drop=True)
        # get average concentration
        avg_m = metal['Actual'].mean()
        avg_metal.append(avg_m)
        # get R2
        r2_m = r2_score(metal.Actual, metal.Pred)
        r2_metal.append(r2_m)
        # get RMSE-P
        metal['sqerror'] = (metal.Actual - metal.Pred).pow(2)
        rmse_m = metal['sqerror'].mean() ** 0.5
        rmsep_metal.append(rmse_m)

  0%|          | 0/37 [00:00<?, ?it/s]

#### Show results

In [134]:
instr = [instrument] * len(elem)
atmos = [atm] * len(elem)
num = [num_range] * len(elem)

temp = pd.DataFrame({
    "element" : elem,
    'instrument' : instr,
    'atmosphere' : atmos,
    'num_range' : num,
    "Avg_Braga" : avg_braga,
    "Avg_metals" : avg_metal,
    "RMSEP_Braga" : rmsep_braga,
    "RMSEP_metals" : rmsep_metal,
    "R2_Braga" : r2_braga,
    "R2_metals" : r2_metal
})

display(temp)

,element,instrument,atmosphere,num_range,Avg_Braga,Avg_metals,RMSEP_Braga,RMSEP_metals,R2_Braga,R2_metals
0,Li (ppm),LANL,Mars,0-750,60.575000,39.068421,9.649220,8.792369,0.460190,0.753752
1,MnO (wt%),LANL,Mars,0-750,0.149308,0.149627,0.276573,0.275669,0.021523,0.017796
2,Na2O (wt%),LANL,Mars,0-750,2.385991,2.198115,0.642265,0.618024,0.843797,0.863595
3,Ni (ppm),LANL,Mars,0-750,80.224490,80.019231,83.243927,83.148719,0.161145,0.130616
4,Pb (ppm),LANL,Mars,0-750,13.435714,13.435714,14.293521,14.293521,-0.217646,-0.217646
5,Rb (ppm),LANL,Mars,0-750,104.880645,84.050000,45.795482,40.776928,0.443082,0.625541
6,SiO2 (wt%),LANL,Mars,0-750,57.580648,55.236854,5.780683,5.980671,0.802905,0.867307
7,Sr (ppm),LANL,Mars,0-750,457.500000,251.715625,103.314595,87.852496,0.354837,0.673886
8,Zn (ppm),LANL,Mars,0-750,104.992754,104.407143,77.781744,77.501411,0.282187,0.279077


### Add to results table

In [135]:
full_table = full_table.append(temp).reset_index(drop = True)
# drop unit info
full_table.element = full_table.apply(lambda row: row.element.split()[0], axis = 1)
display(full_table)

,element,instrument,atmosphere,num_range,Avg_Braga,Avg_metals,RMSEP_Braga,RMSEP_metals,R2_Braga,R2_metals
0,Li,LANL,Mars,250-1000,35.000000,32.100000,28.487200,12.274117,NaN,-0.523450
1,MnO,LANL,Mars,250-1000,0.095000,0.094946,0.089460,0.089387,-0.188372,-0.199565
2,Na2O,LANL,Mars,250-1000,2.325412,2.152657,0.686565,0.658986,0.818104,0.839089
3,Ni,LANL,Mars,250-1000,83.298611,83.298611,83.366447,83.366447,0.332041,0.332041
4,Pb,LANL,Mars,250-1000,20.955102,20.955102,18.806932,18.806932,-0.016758,-0.016758
5,Rb,LANL,Mars,250-1000,110.828261,102.892000,30.983942,29.994633,0.729968,0.775791
6,SiO2,LANL,Mars,250-1000,57.608847,55.467275,4.332025,6.278819,0.875408,0.840566
7,Sr,LANL,Mars,250-1000,461.833333,303.932258,151.424187,106.398020,-0.224944,0.569745
8,Zn,LANL,Mars,250-1000,84.877193,80.709677,48.834792,47.870917,-0.002031,0.078121
9,Li,LANL,Mars,0-750,60.575000,39.068421,9.649220,8.792369,0.460190,0.753752


### Export full table

In [136]:
epath = "C:\\Users\\ytsma22c\\Google Drive\\Misc Work\\LOD paper\\test_calcs.csv"
full_table.to_csv(epath, index=False)